# <center><font color = "green">TWITTER SENTIMENT ANALYZER</font></center>

<center>
  <img src="https://cdn270.picsart.com/4bfd63fc-41bb-48a3-a5e7-08074a542160/433348990036201.gif" style="width: 300px; height: 300px;" alt="Image">
</center>


### Table Content :

1. [Importing Libraries](#import)
2. [Exploring Through Data](#explore)
3. [Data Cleaning](#clean)
4. [Tfidf vectorizer](#tfidf)

# 1.) <font color = "black">Importing Libraries</font><a class = "anchor" id = "import"></a>

In [2]:
import pandas as pd
import re

In [3]:
train_data = pd.read_csv(r"C:\Users\CHAITANYA\Documents\SENTIMENTAL_ANALYSIS_NLP_PROJECT\Train__Data.csv")

test_data = pd.read_csv(r"C:\Users\CHAITANYA\Documents\SENTIMENTAL_ANALYSIS_NLP_PROJECT\Test_Data.csv")

In [4]:
train_data.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,Just published a new blog post on AI technolog...,new blog technology,neutral
1,549e992a42,Excited to share my latest recipe creation: Tr...,excited latest recipe,positive
2,3b9a1f0c2e,The bird flies high in the sky. #nature #peace,flies high in the sky,neutral
3,8d7c5e4f6a,What a thrilling cricket match today! Nail-bit...,thrilling Nail-biting finish last ball,positive
4,088c60f138,"""Late-night studies can be tough, but it`s all...",can be tough,negative


In [5]:
test_data.head()

,textId,text,selected_text,sentiment
0,4234ca4536,Lookin` at spreadsheets so long my eyes are cr...,my eyes are crossing...,negative
1,181bc1ee99,no way - those are great words. Boys don`t g...,no way - those are great words.,neutral
2,ee60f31d77,/drool. I still need to 100% the first one,/drool. I still need to 100% the first one,neutral
3,9457c9ad0f,Word. Yayy twitter after dark lol.,. Yayy,positive
4,54f64bce62,"So, #primavera is in Barcelona. In Spain. Sure...","So, #primavera is in Barcelona. In Spain. Sure...",neutral


# 2.)  <font color = "black">Exploring Through Data</font><a class = "anchor" id = "explore"></a>

In [6]:
train_data.isnull().sum()

textID           0
text             1
selected_text    1
sentiment        0
dtype: int64

In [7]:
test_data.isnull().sum()

textId           0
text             0
selected_text    0
sentiment        0
dtype: int64

In [9]:
train_data.dropna(inplace = True)
train_data.isnull().sum()

textID           0
text             0
selected_text    0
sentiment        0
dtype: int64

In [74]:
def rename_column(dataset):
    dataset.rename(columns = { '  TEXT' : 'tweet' } , inplace = True)
    return dataset

rename_column(train_data)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded to 5 mails , if I "" we...","I`d have responded, if I were going",neutral
1,549e992a42,"Sooo SAD @siva , * I will miss you ...",Sooo SAD,negative
2,088c60f138,"I am sry , my boss is bullying #ceasefire a...",bullying me,negative
3,9642c003ef,what interview awsm! leave me alone dnd for ...,leave me alone,negative
4,358bd9e861,enuf It is very awks to see bcz they insulted him,insulted,negative
...,...,...,...,...
9994,dae1f9f100,im 16 too,im 16 too,neutral
9995,9e6bab2198,Just got bullied by Dillah. HELP!,bullied,negative
9996,29ffa4dfad,what lol?,what lol?,neutral
9997,4eb36fc6ac,Some days you must look hard for that good th...,you must look hard for that good thing.,positive


In [75]:
train_data.columns

Index(['textID', 'text', 'selected_text', 'sentiment'], dtype='object')

In [76]:
train_data["Length"] = train_data["text"].str.len()

train_data["Words"] = train_data["text"].str.split().str.len()

In [77]:
display(train_data[["Length" , "Words"]])

,Length,Words
0,73.0,16.0
1,101.0,21.0
2,62.0,12.0
3,55.0,11.0
4,49.0,11.0
...,...,...
9994,10.0,3.0
9995,34.0,6.0
9996,10.0,2.0
9997,50.0,10.0


In [78]:
# REPEATED TWEETS
train_data.groupby(['text']).size().reset_index(name='counts')\
  .sort_values('counts', ascending=False).head(10)

,text,counts
0,"Sooo SAD @siva , * I will miss you ...",1
6668,SCENE **** AT 4AM IS ALWAYS THE ****,1
6661,"Rope witdrawal in full effect, no one to tie, ...",1
6662,Roscoe - Smooth Sailin` WHY DID NO ONE TELL ME...,1
6663,"Rose Romance is released here on Thursday, gon...",1
6664,Round and Round the Garden: The amazing thing ...,1
6665,Ryan Stiles is still the funniest man ever. Go...,1
6666,SB 239K. Flop A Q 9. 4 players check. Turn 9...,1
6667,SBS 2003 or SBS 2008! Damned if you do and dam...,1
6669,SEe waT I Mean bOuT FoLL0w fRiiDaYs... It`S cA...,1


# 3.) <font color = "black">Data Cleaning</font><a class = "anchor" id = "clean"></a>

In [79]:
def rename_column(dataset):
    dataset.rename(columns = { 'text' : 'tweet' } , inplace = True)
    return dataset

rename_column(train_data)
rename_column(test_data)
train_data.head()

,textID,tweet,selected_text,sentiment,Length,Words
0,cb774db0d1,"I`d have responded to 5 mails , if I "" we...","I`d have responded, if I were going",neutral,73.0,16.0
1,549e992a42,"Sooo SAD @siva , * I will miss you ...",Sooo SAD,negative,101.0,21.0
2,088c60f138,"I am sry , my boss is bullying #ceasefire a...",bullying me,negative,62.0,12.0
3,9642c003ef,what interview awsm! leave me alone dnd for ...,leave me alone,negative,55.0,11.0
4,358bd9e861,enuf It is very awks to see bcz they insulted him,insulted,negative,49.0,11.0


In [80]:
encoding_emotion = {"positive" : 1 , "negative" : -1 , "neutral" : 0}

In [81]:
def encoding_emotions(dataset):
    dataset.replace(encoding_emotion, inplace = True)
    return dataset

In [82]:
encoding_emotions(train_data)
encoding_emotions(test_data)

,textId,tweet,selected_text,sentiment
0,4234ca4536,Lookin` at spreadsheets so long my eyes are cr...,my eyes are crossing...,-1
1,181bc1ee99,no way - those are great words. Boys don`t g...,no way - those are great words.,0
2,ee60f31d77,/drool. I still need to 100% the first one,/drool. I still need to 100% the first one,0
3,9457c9ad0f,Word. Yayy twitter after dark lol.,. Yayy,1
4,54f64bce62,"So, #primavera is in Barcelona. In Spain. Sure...","So, #primavera is in Barcelona. In Spain. Sure...",0
...,...,...,...,...
1996,6c86c96dbe,I`m missing one of my diamond earrings. This m...,I`m missing one of my diamond earrings. This m...,-1
1997,fbfad74229,"I love wearing my hood too, even though in Po...","I love wearing my hood too, even though in Por...",0
1998,948e5f7b25,Well I`m obsessed w/all of them - but I watch...,Well I`m obsessed w/all of them - but I watche...,0
1999,6021b57412,is this a new album?? about bloody time too!!,is this a new album?? about bloody time too!!,0


In [83]:
train_data.head()

,textID,tweet,selected_text,sentiment,Length,Words
0,cb774db0d1,"I`d have responded to 5 mails , if I "" we...","I`d have responded, if I were going",0,73.0,16.0
1,549e992a42,"Sooo SAD @siva , * I will miss you ...",Sooo SAD,-1,101.0,21.0
2,088c60f138,"I am sry , my boss is bullying #ceasefire a...",bullying me,-1,62.0,12.0
3,9642c003ef,what interview awsm! leave me alone dnd for ...,leave me alone,-1,55.0,11.0
4,358bd9e861,enuf It is very awks to see bcz they insulted him,insulted,-1,49.0,11.0


In [84]:
test_data.head()

,textId,tweet,selected_text,sentiment
0,4234ca4536,Lookin` at spreadsheets so long my eyes are cr...,my eyes are crossing...,-1
1,181bc1ee99,no way - those are great words. Boys don`t g...,no way - those are great words.,0
2,ee60f31d77,/drool. I still need to 100% the first one,/drool. I still need to 100% the first one,0
3,9457c9ad0f,Word. Yayy twitter after dark lol.,. Yayy,1
4,54f64bce62,"So, #primavera is in Barcelona. In Spain. Sure...","So, #primavera is in Barcelona. In Spain. Sure...",0


In [85]:
col_drop = ['Length' , 'Words' ,'textID' , 'selected_text' ]
colu_drop = ['textId' , 'selected_text' ]


test_data = test_data.drop(columns = colu_drop)
train_data = train_data.drop(columns = col_drop)

In [86]:
# Seperating features and labels

def separate_feature_labels(dataset):
  features =   dataset["tweet"].to_numpy()
  labels   =   dataset["sentiment"].to_numpy()
  return (features, labels)

In [87]:
(train_features, train_labels) = separate_feature_labels(train_data)

(test_features, test_labels) = separate_feature_labels(test_data)

In [88]:
train_features

array(['    I`d have responded to 5 mails , if I "  were going kwl ! <br> \\n     ',
       '          Sooo SAD @siva ,  * I will miss you here in "San Diego for at least 10 days!!! <br /><br />',
       '   I am sry , my boss is bullying #ceasefire and @prabhas <br>',
       ..., ' what lol?',
       ' Some days you must look hard for that good thing.',
       'i`m so worn out, this week has sped by soo fast. gonna ask mum if we can go to the movies. i`m desperate'],
      dtype=object)

In [89]:
train_labels

array([ 0, -1, -1, ...,  0,  1, -1], dtype=int64)

### NLP Techniques :

1.) [Lower Casing](#lower)<br>
2.) [Removing Duplicates](#duplicate)<br>
3.) [Text Normalization](#normalize)<br>
4.) [Tokentization](#tokenize)<br>
5.) [Drop Numbers](#numbers)<br>
6.) [Stopwords Removal](#stopwords)<br>
7.) [Using Re](#Re)<br>
8.) [Lemmatization](#lemmatize)

#  1.) <font color = "black">Lower Casing</font><a class = "anchor" id = "lower"></a>

In [90]:
# if x is string => apply x.lower()
# if x is not string return x

train_data['tweet'] = train_data['tweet'].apply(lambda x: x.lower() if isinstance(x, str) else x)
test_data['tweet'] = test_data['tweet'].apply(lambda x: x.lower() if isinstance(x, str) else x)
train_data['tweet']

0           i`d have responded to 5 mails , if i "  we...
1                 sooo sad @siva ,  * i will miss you ...
2          i am sry , my boss is bullying #ceasefire a...
3        what  interview awsm! leave me alone dnd for ...
4       enuf it is very awks to see bcz they insulted him
                              ...                        
9994                                            im 16 too
9995                   just got bullied by dillah.  help!
9996                                            what lol?
9997     some days you must look hard for that good th...
9998    i`m so worn out, this week has sped by soo fas...
Name: tweet, Length: 9999, dtype: object

# 2.) <font color = "black">Removing Duplicates</font><a class = "anchor" id = "duplicate"></a>

In [91]:
# keep = false : if a tweet is duplicate then it's 1st occurence also considered as duplicate

duplicates1 = train_data[train_data['tweet'].duplicated(keep=False)]
duplicates2 = test_data[test_data['tweet'].duplicated(keep=False)]


In [92]:
print(duplicates1)

                                            tweet  sentiment
95    happy mothers day to all you mums out there          1
170                                  good morning          1
1139                           happy mothers day!          1
1387                                       thanks          1
1643                                   thank you.          1
1788                                 good morning          1
2175                                       thanks          1
2354                                  tummy hurts         -1
2408                                 good morning          1
2932                                    thank you          1
3274                                    thank you          1
3975                           happy mothers day!          1
4114                                 good morning          1
4413                               i miss you too         -1
6532                               i miss you too         -1
7424                    

In [93]:
print(duplicates2)

Empty DataFrame
Columns: [tweet, sentiment]
Index: []


In [94]:
# Removing duplicates

# keep='first' => retains 1st occurence of duplicate value and remaining(subsequent) duplicates are dropped

train_data.drop_duplicates(subset='tweet', keep='first', inplace=True)
test_data.drop_duplicates(subset='tweet', keep='first', inplace=True)
print(train_data['tweet'])

0           i`d have responded to 5 mails , if i "  we...
1                 sooo sad @siva ,  * i will miss you ...
2          i am sry , my boss is bullying #ceasefire a...
3        what  interview awsm! leave me alone dnd for ...
4       enuf it is very awks to see bcz they insulted him
                              ...                        
9994                                            im 16 too
9995                   just got bullied by dillah.  help!
9996                                            what lol?
9997     some days you must look hard for that good th...
9998    i`m so worn out, this week has sped by soo fas...
Name: tweet, Length: 9989, dtype: object


In [95]:
duplicates = train_data[train_data['tweet'].duplicated(keep=False)]

print("Duplicated Sentences:")
print(duplicates)

Duplicated Sentences:
Empty DataFrame
Columns: [tweet, sentiment]
Index: []


# 3.) <font color = "black">Text Normalization</font><a class = "anchor" id = "normalize"></a>

In [96]:
appostrophy_dict = {
    "can`t": "cannot",
    "cant": "cannot",
    "aren`t": "are not",
    "arent": "are not",
    "couldn`t": "could not",
    "couldnt": "could not",
    "doesn`t": "does not",
    "doesnt": "does not",
    "don`t": "do not",
    "dont": "do not",
    "hadn`t": "had not",
    "hadnt": "had not",
    "hasn`t": "has not",
    "hasnt": "has not",
    "haven`t": "have not",
    "havent": "have not",
    "he`d": "he would",
    "he`ll": "he will",
    "he`s": "he is",
    "i`d": "i would",
    "i`ll": "I will",
    "i`m": "i am",
    "im": "i am",
    "isn`t": "is not",
    "isnt": "is not",
    "it`s": "it is",
    "it`ll": "it will",
    "i`ve": "i have",
    "let`s": "let us",
    "mightn`t": "might not",
    "mightnt": "might not",
    "mustn`t": "must not",
    "mustnt": "must not",
    "shan`t": "shall not",
    "she`d": "she would",
    "she`ll": "she will",
    "she`s": "she is",
    "shouldn`t": "should not",
    "shouldnt": "should not",
    "that`s": "that is",
    "thats": "that is",
    "there`s": "there is",
    "they`d": "they would",
    "they`ll": "they will",
    "they`re": "they are",
    "they`ve": "they have",
    "we`d": "we would",
    "we`re": "we are",
    "weren`t": "were not",
    "we`ve": "we have",
    "what`ll": "what will",
    "what`re": "what are",
    "what`s": "what is",
    "whats": "what is",
    "what`ve": "what have",
    "where`s": "where is",
    "who`d": "who would",
    "who`ll": "who will",
    "who`re": "who are",
    "who`s": "who is",
    "who`ve": "who have",
    "won`t": "will not",
    "wont": "will not",
    "wouldn`t": "would not",
    "wouldnt": "would not",
    "you`d": "you would",
    "you`ll": "you will",
    "you`re": "you are",
    "you`ve": "you have",
    "wasn`t": "was not",
    "wasnt": "was not",
    "we`ll": "will",
    "didn`t": "did not",
    "didnt": "did not"
}

def removingAppostrophies(tokens):
    if tokens is not None:
        removedText = []
        for token in tokens:
            token_lower = token.lower()
            if token_lower.endswith("`s"):
                removedText.append(token_lower[:-2] + " is")
            elif token_lower in appostrophy_dict:
                removedText.append(appostrophy_dict[token_lower])
            else:
                removedText.append(token)
        value = " ".join(removedText)
        return value

train_data['tokenized_tweet'] = train_data['tweet'].apply(lambda x: x.split() if isinstance(x, str) else [])
train_data['tokenized_tweet'] = train_data['tokenized_tweet'].apply(removingAppostrophies)


print(train_data['tokenized_tweet'])

0       i would have responded to 5 mails , if i " wer...
1       sooo sad @siva , * i will miss you here in "sa...
2       i am sry , my boss is bullying #ceasefire and ...
3       what interview awsm! leave me alone dnd for 10...
4       enuf it is very awks to see bcz they insulted him
                              ...                        
9994                                          i am 16 too
9995                    just got bullied by dillah. help!
9996                                            what lol?
9997    some days you must look hard for that good thing.
9998    i am so worn out, this week has sped by soo fa...
Name: tokenized_tweet, Length: 9989, dtype: object


# 4.) <font color = "black">Tokenization</font><a class = "anchor" id = "tokenize"></a>

In [97]:
def tokenization(text):

    punctuation = '!"$%&\()*+,-./:;<=>?[\\]^_`{|}~'
    if isinstance(text, str):
        for char in punctuation:
            text = text.replace(char, ' ')
        tokens = text.split()

        return tokens

train_data['tokenized_tweet']=train_data['tokenized_tweet'].apply(tokenization)
print(train_data['tokenized_tweet'])

0       [i, would, have, responded, to, 5, mails, if, ...
1       [sooo, sad, @siva, i, will, miss, you, here, i...
2       [i, am, sry, my, boss, is, bullying, #ceasefir...
3       [what, interview, awsm, leave, me, alone, dnd,...
4       [enuf, it, is, very, awks, to, see, bcz, they,...
                              ...                        
9994                                     [i, am, 16, too]
9995               [just, got, bullied, by, dillah, help]
9996                                          [what, lol]
9997    [some, days, you, must, look, hard, for, that,...
9998    [i, am, so, worn, out, this, week, has, sped, ...
Name: tokenized_tweet, Length: 9989, dtype: object


In [98]:
word_counts = {}
for tokenized_list in train_data['tokenized_tweet']:
    
    # Count word frequencies in this row
     for word in tokenized_list:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

# Print word frequencies
for word, count in word_counts.items():
    print(f'{word}: {count}')

i: 6135
would: 256
have: 1132
responded: 1
to: 3823
5: 87
mails: 2
if: 270
were: 134
going: 443
kwl: 1
br: 5
n: 81
sooo: 52
sad: 183
@siva: 1
will: 683
miss: 231
you: 1968
here: 250
in: 1375
san: 9
diego: 5
for: 1371
at: 691
least: 58
10: 58
days: 125
am: 1401
sry: 5
my: 1972
boss: 14
is: 2898
bullying: 1
#ceasefire: 1
and: 1932
@prabhas: 1
what: 406
interview: 7
awsm: 1
leave: 57
me: 1098
alone: 28
dnd: 2
min: 9
enuf: 1
it: 1891
very: 181
awks: 1
see: 317
bcz: 1
they: 260
insulted: 1
him: 132
http: 432
www: 48
dothebouncy: 1
com: 325
smf: 1
some: 322
shameless: 1
plugging: 2
the: 3344
best: 114
rangers: 1
forum: 4
on: 1047
earth: 11
hbbd: 1
hero: 6
hf: 1
let's: 1
connect: 3
fb: 13
dm: 10
hwu: 1
idt: 1
ihu: 1
that: 1217
particular: 2
topic: 6
hey: 109
sup: 1
much: 257
str: 1
journey: 5
wow: 81
u: 370
just: 818
became: 5
cooler: 3
lemeno: 1
as: 246
love: 407
be: 744
hopeful: 2
reckon: 2
chances: 3
are: 735
minimal: 1
p: 78
never: 120
gonna: 156
get: 502
cake: 24
stuff: 57
really: 322
li

worth: 21
shot: 9
opened: 2
confused: 10
seems: 32
ship: 5
dustbin: 2
11: 20
30: 48
dwell: 1
happened: 21
passed: 9
shame: 14
supportive: 1
office: 45
tv: 45
mlb: 1
mlbn: 1
televising: 1
det: 1
bal: 1
#wieters: 1
empty: 13
handed: 2
comics: 4
found: 63
shall: 14
indulge: 2
cupcakes: 8
magnolia: 1
bakery: 3
laying: 11
workkk: 1
definitely: 29
pinched: 2
nerve: 1
exhausting: 3
weekend: 158
ur: 107
flight: 12
cancelled: 8
blows: 4
ohh: 16
distracted: 4
taking: 36
trouble: 7
follow: 84
others: 15
feelin: 11
bills: 5
ha: 51
ii: 7
paying: 6
y: 32
broke: 29
sure: 85
jst: 4
planning: 12
tet: 1
outing: 1
marwell: 1
times: 44
mcfly: 11
gig: 11
nightt: 1
amazin: 1
sit: 13
ma: 10
mate: 16
tutus: 1
without: 57
migraine: 5
postr01: 1
relaxing: 20
whens: 2
sway: 2
winner: 3
announced: 6
k: 15
wide: 8
awake: 25
wake: 30
busyy: 1
booo: 9
whoo: 3
mommy: 29
dog: 34
officially: 14
dogs: 13
half: 23
heartedly: 1
realized: 16
msgs: 1
frist: 1
parents: 24
exciting: 18
argh: 9
noo: 2
killers: 3
wossy: 2
brand

hhahaa: 1
ahah: 2
lolllllyyyyyyyy: 1
nexus: 1
suitcase: 1
bloody: 8
indecicive: 1
everytime: 4
takes: 15
pleaseeeeeeeeeee: 1
sum: 5
serious: 14
prevent: 1
bites: 3
itchy: 1
distracting: 1
editing: 3
curb: 1
flat: 7
tire: 3
stood: 3
omnomnom: 1
69: 1
unread: 2
emails: 7
backlog: 1
lxjdjc: 1
hook: 2
rian: 1
van: 5
staden: 1
concrete: 2
nako: 1
umuulan: 1
pa: 3
naman: 1
oohh: 1
yaay: 2
dota: 2
addiction: 2
waste: 6
able: 40
ff: 7
frisbee: 1
golfer: 1
'accidentally': 1
catches: 1
lhoste: 1
da: 22
usael: 1
notmuch: 1
woman: 8
momma: 6
smoking: 3
cig: 1
chiodos: 1
elk: 1
grove: 1
village: 4
ordered: 7
unexpected: 2
chase: 2
richard: 2
dumb: 6
smart: 2
memo: 1
ppl: 22
goooooood: 1
hilariouss: 1
loveeee: 5
theree: 1
grateful: 3
burn: 6
twee: 2
temporary: 3
guy: 34
batteries: 2
besides: 6
vampire: 1
touched: 3
changi: 1
hrs: 9
penang: 1
singapore: 4
eff: 1
chris: 8
browning: 1
positivity: 2
regret: 5
forsure: 2
sent: 18
bck: 1
credit: 7
thts: 1
mint: 4
dessert: 2
snl: 18
ucsd: 1
blackberry: 14


sleeeeeppyyyyyy: 1
cos: 13
sacred: 1
reality: 4
hosp: 1
jams: 3
rubbish: 4
a48: 1
cowbridge: 1
cardiff: 1
directions: 1
kay: 4
bccg: 1
download: 8
'still: 1
d9g8sj: 1
#movie: 1
jacksonville: 1
dishwasher: 1
meets: 1
fau: 1
steet: 1
slowmotion: 1
insight: 1
stunned: 1
aweomse: 1
robluketic: 2
ustre: 8
2w5v: 2
entire: 8
breath: 5
possessed: 1
certainty: 1
decision: 2
brooke: 2
hyper: 6
girly: 3
omds: 1
steel: 5
polar: 1
bears: 1
christian: 8
lacroix: 2
clients: 4
bust: 6
kv653j: 1
pressure: 3
situation: 7
meds: 2
unfuzzy: 1
prefer: 6
fuzzy: 1
clear: 6
caleb: 1
comin: 4
2morrow: 4
tennis: 3
nar: 1
yuppie: 1
forums: 3
leader: 1
surely: 3
il: 2
divo: 1
insurance: 4
devil: 4
widow: 1
tattoo: 5
mike: 3
kinf: 1
contact: 11
touch: 3
literally: 4
junk: 3
yess: 4
amsterdam: 2
18th: 2
april: 2
everr: 1
mooorrreeee: 1
filipino: 3
prof: 1
gorayeb: 1
alreadt: 1
'fuel': 1
braclets: 1
5jg16: 1
infection: 4
strong: 5
taste: 8
greatings: 1
planet: 2
shannan: 1
laded: 1
lanet: 1
hopefull: 1
haiaa: 1
finni

saddens: 1
dc: 5
07am: 1
ugghhh: 1
#: 5
delhi: 1
wzk3o: 1
paranoid: 4
performed: 1
lumberjack: 1
opendns: 1
crappy: 8
dns: 2
par: 1
appointment: 1
whit: 1
'bus: 1
stalking': 1
saynow: 2
yayy: 4
jbs: 1
canadian: 1
landline: 1
remembered: 5
operational: 2
chill: 8
rihanna: 1
concentrate: 2
harsh: 1
savvy: 1
tucson: 1
miserable: 5
k3gmw: 1
shuts: 1
transfer: 3
impressions: 1
sexual: 1
maternal: 1
threatening: 1
themselves: 3
weak: 4
fog: 1
pearl: 2
harbor: 1
branding: 1
iron: 4
drea: 1
boubous: 1
crunk: 2
hailee: 1
possibly: 4
lafayette: 1
updating: 4
yew: 4
reflecting: 1
sunlight: 1
rolled: 1
useless: 3
crib: 2
abt: 7
scenic: 1
oyster: 1
cult: 2
5jd4b: 1
inviting: 1
phatass: 1
passing: 3
maryland: 1
camden: 1
trousers: 3
loose: 4
tight: 3
sniffles: 2
rof: 1
eep: 1
fangirl: 2
hollowbabeshere: 1
utter: 1
shite: 1
bffls: 1
nevah: 1
tells: 3
hahahahaha: 3
shape: 2
via: 13
stale: 2
sweets: 1
reminds: 3
upto: 1
xoxox: 1
classics: 2
eather: 1
wabble: 2
enterprise: 2
girlfriend: 4
sesame: 1
wade

waaaah: 1
reeesee: 1
comet: 1
arrives: 2
12p: 1
havnt: 3
gotten: 5
dating: 1
spark: 1
loser: 5
nothings: 1
isolated: 1
everyoneeeee: 1
handsome: 1
elaine: 3
mines: 7
twitta: 2
southridge: 1
peacin: 1
aisa: 1
shifty: 1
regardless: 2
cubes: 1
vacationing: 1
dreamt: 1
monsters: 1
lo: 3
siento: 1
dint: 1
baristas: 1
commonalities: 1
alwayyyyyyyssssssss: 1
mcdonalds: 3
surpisingly: 1
diners: 1
24hrs: 2
frat: 1
thxs: 1
afterwards: 4
'its: 1
guarantee: 1
fruit: 3
basket: 1
pooooor: 1
todayyyyy: 1
archie: 2
proposes: 1
veronica: 2
comic: 2
longest: 2
triangle: 1
betty: 2
cringe: 2
pause: 1
proves: 1
blahh: 2
kiddies: 1
speakerphone: 1
qxyc2n: 1
seos: 1
submitting: 1
information: 2
lly: 1
keynote: 1
#screencastsonline: 1
nicky: 2
desperation: 1
66zls: 1
humidity: 1
majorly: 2
hhhmmmnnn: 1
tanner: 1
concealer: 1
wise: 3
coooolest: 1
nichole: 1
earrings: 1
von: 1
abe: 1
dfggwj: 1
suprisingly: 1
overall: 2
fails: 2
deplurk: 1
buhbyeee: 1
rp3ir: 1
creeped: 1
din: 1
dins: 1
megaredpacket: 1
mega: 1


persist: 1
crazyangel: 1
gregg: 1
sorrry: 1
flasher: 1
romo: 1
sink: 2
cupboard: 1
safety: 1
locks: 1
cullen26: 1
woodstock: 1
frined: 1
77: 1
letters: 1
nfg: 1
responsabilities: 1
bbcrew: 1
whhaacck: 1
pokes: 1
#gm: 1
bankruptcy: 1
offset: 1
gains: 1
tax: 2
vodafone: 1
ads: 2
picturisation: 1
ratatat: 1
gjqsh: 1
profitable: 1
5th: 5
drums: 2
sets: 1
yupp: 1
dragonball: 1
safely: 1
attics: 2
robs: 1
galore: 1
aaaw: 1
heylo: 1
johnn: 1
cuzz: 1
socialscope: 1
gihac: 1
ships: 1
photography: 3
dfs7fy: 1
any1: 1
lying: 3
4fzo7: 1
beatiful: 1
livechat: 1
cus: 1
weekdays: 1
floyd: 1
buena: 1
defend: 1
enemy: 2
excercise: 1
shemar: 1
moore: 1
embassy: 1
tryin: 2
ontario: 1
6789998212: 1
soulja: 3
4wrxq: 1
gingg: 1
proverbs: 2
beauties: 1
grads: 1
4we5w: 1
therefore: 1
wasting: 1
deodorant: 1
funk: 1
duhhhhhh: 1
3am: 2
twin: 2
weezy: 1
knotts: 2
evn: 1
waaah: 2
masseuse: 1
venti: 1
pumps: 1
rhonda: 1
translation: 2
spat: 1
balanced: 1
midge: 1
tie: 3
punk: 1
travels: 2
pin: 2
dandy: 1
cheery: 1

ne: 3
haters: 2
kelli: 1
byron: 1
candid: 1
ness: 1
notebook: 1
filling: 3
seventeen: 1
chorleywood: 1
cambridge: 1
tuning: 1
raising: 1
filas: 1
definently: 1
kimba: 1
certain: 3
groups: 2
juniors: 1
korea: 3
knw: 2
wiv: 1
4yr: 1
standard: 1
blu: 1
drama: 3
seriuosly: 1
reminders: 1
posture: 1
amanda: 1
peet: 1
fields: 1
workweek: 1
sim: 1
mindless: 1
walwal: 1
core: 4
ers: 1
youre: 5
sumptuous: 1
matta: 1
'how: 1
yday: 1
tallebudgera: 1
anita: 1
spite: 1
unforgettable: 1
pwns: 2
xtra: 1
wolvarine: 1
frnd: 1
lh: 1
fopp: 1
damnnn: 1
timeee: 1
bahhhh: 1
athens: 2
'sicko': 1
utterly: 1
digusted: 1
importantly: 2
buttfuck: 1
forgetful: 1
ssshh: 1
empitome: 1
sharapova: 1
unseeded: 1
qualifier: 1
doubles: 2
wellity: 1
sensibility: 1
corrupted: 1
cap: 4
bidden: 1
hip: 2
hop: 1
bseresults: 1
treadmill: 1
ucl: 1
install: 3
imacs: 1
digiqom: 1
welcomed: 1
98: 2
studyhall: 1
bach: 2
contributions: 1
silent: 2
linux: 1
definitley: 1
recos: 1
animating: 1
stat: 2
jkuk: 1
thaz: 1
riot: 1
luton: 1


#justin: 1
#mark: 1
#4: 1
mocking: 1
partytiiime: 1
medisoft: 1
fiesta: 1
twitternut: 1
squashes: 1
mozy: 1
scruffy: 1
nerfherders: 1
ami: 2
4wsy6: 1
beauty: 2
spree: 2
extension: 1
contract: 1
kari: 1
jazzy: 2
pony: 1
sumone: 1
fletcher: 1
twins: 2
dammm: 1
lollllll: 1
knowledgable: 1
france: 1
mpre: 1
chanclas: 1
swedish: 3
noe: 2
hmmmmmmm: 1
aiaahh: 1
judah: 1
roscoe: 1
sailin: 1
liver: 1
struggles: 1
forgets: 1
aid: 1
excitement: 1
iwish: 1
board: 2
15minutes: 1
racers: 1
pineapple: 1
smoothie: 2
logs: 1
facepanda: 1
flooding: 1
'philippines: 1
allegra: 1
flonase: 1
steroid: 1
inhaler: 1
allergy: 2
chore: 1
mopping: 1
pune: 1
milkshakes: 1
areo: 1
blended: 1
lumpy: 1
chattering: 1
freebies: 1
jbnoy: 1
cullen8: 1
300: 2
22nd: 1
helen: 1
vmware: 1
lolz: 1
rels: 1
beggin: 1
notices: 2
hannahmovie: 1
withmom: 1
johny: 1
ironic: 1
producing: 1
anchoring: 1
lynne: 1
ashminov: 1
zion: 1
pure: 1
genious: 1
falls: 1
1pm: 1
collections: 1
youth: 2
wonderfull: 1
breakfst: 1
ofcourse: 1
chemis

amything: 1
sence: 1
lover: 1
scratching: 1
mueller: 1
gromit: 1
bbc1: 1
goooood: 1
tricked: 1
hype: 1
mcm: 1
sprigge: 1
skydivers: 1
doomsday: 1
guyz: 1
doinq: 1
eligable: 1
howeva: 1
itz: 1
spelled: 1
paniniii: 1
cramming: 1
chf: 1
670: 1
mashup: 1
hysteria: 1
5jnv2: 1
haul: 1
wagner: 1
bend: 1
shz: 1
5ke: 1
employee: 1
'hide: 1
james': 1
530p: 1
areply: 1
mmmmmmm: 1
espresso: 1
efficiently: 1
fur: 2
cosmic: 1
irony: 1
furry: 1
giveaway: 1
unclaimed: 1
outdated: 2
bigoted: 1
patronising: 1
imperialist: 1
monoculturalist: 1
righteous: 1
congratses: 1
tew: 1
frosting: 1
tara: 1
catholic: 1
wholeheartedly: 1
'same: 1
here': 1
ambiguous: 1
#hhrs: 2
bandit: 1
#deadliestcatch: 1
tpg: 1
suspense': 1
whatnot: 1
godawful: 1
gahd: 1
troopers: 1
uggo: 1
excuses: 1
'17: 1
again': 1
catnapped: 1
oberhausen: 1
girlz: 1
jess: 2
vague: 1
popper: 1
registration: 1
suxs: 1
pestered: 1
personnel: 1
mumbai: 1
yardhouse: 1
waikiki: 2
breakie: 1
blackpool: 1
flowered: 1
pose: 1
imitate: 1
shaycarl: 1
vlog

In [99]:
#removing urls from tweets
#Tweets_Cleaned=re.sub(r'http\S+', ' ', Tweets_Cleaned)

#removing "usernames" from tweets
#Tweets_Cleaned=re.sub(r'@\w+', ' ', Tweets_Cleaned)

#removing everything except "charecters"
#Tweets_Cleaned = re.sub(r'[^a-z A-Z]', ' ',Tweets_Cleaned)

#removing "extra spaces" from tweets
#Tweets_Cleaned= re.sub(r' +', ' ', Tweets_Cleaned)


# 5.) <font color = "black">Drop Numbers</font><a class = "anchor" id = "numbers"></a>

In [100]:
def RemovingNumbers(tokens):
    if tokens is None:
        return []
    else:
        pattern = r'\b\w*\d\w*\b'
        removingNumbers_tokens = [token for token in tokens if not re.search(pattern, token)]
        return removingNumbers_tokens

 
train_data["tokenized_tweet"] = train_data["tokenized_tweet"].apply(RemovingNumbers)
print(train_data["tokenized_tweet"])

0       [i, would, have, responded, to, mails, if, i, ...
1       [sooo, sad, @siva, i, will, miss, you, here, i...
2       [i, am, sry, my, boss, is, bullying, #ceasefir...
3       [what, interview, awsm, leave, me, alone, dnd,...
4       [enuf, it, is, very, awks, to, see, bcz, they,...
                              ...                        
9994                                         [i, am, too]
9995               [just, got, bullied, by, dillah, help]
9996                                          [what, lol]
9997    [some, days, you, must, look, hard, for, that,...
9998    [i, am, so, worn, out, this, week, has, sped, ...
Name: tokenized_tweet, Length: 9989, dtype: object


# 6.) <font color = "black">StopWords Removal</font><a class = "anchor" id = "stopwords"></a>

In [101]:
stop_words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 
              'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 
              'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'is', 's', 'am', 'or', 'who', 
              'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 
              'these', 'your', 'his', 'through', 'don', 'me', 'were', 'her', 'more', 'himself', 
              'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to',
              'ours', 'had', 'she', 'all', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 
              'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because',
              'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 
              'herself', 'has', 'just', 'where', 'only', 'myself', 'which', 'those', 'i', 
              'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 
              'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than',
              'ti','ame',' m', 'have' ,'i','say','ames','someti'
              ]

In [102]:
train_data['tokenized_tweet'] = train_data['tokenized_tweet'].apply(lambda tokens: [token for token in tokens if token.lower() not in stop_words])

In [103]:
train_data['tokenized_tweet']

0            [would, responded, mails, going, kwl, br, n]
1       [sooo, sad, @siva, miss, san, diego, least, da...
2         [sry, boss, bullying, #ceasefire, @prabhas, br]
3           [interview, awsm, leave, alone, dnd, min, br]
4                        [enuf, awks, see, bcz, insulted]
                              ...                        
9994                                                [too]
9995                         [got, bullied, dillah, help]
9996                                                [lol]
9997                [days, must, look, hard, good, thing]
9998    [worn, week, sped, soo, fast, gonna, ask, mum,...
Name: tokenized_tweet, Length: 9989, dtype: object

#  Spell check

In [119]:
abbreviation_dict = {
    'awsm': 'awesome',
    'abt': 'about',
    'abt2': 'about to',
    'thr': 'there',
    'frm': 'from',
    'dob': 'date of birth',
    "dnd": "do not disturb",
    'aka': 'also known as',
    'amazn': 'amazing',
    'asap': 'as soon as possible',
    'awks': 'awkward',
    'azm': 'awesome',
    'bcz': 'because',
    'cyb': 'call you back',
    'dis': 'this',
    'enuf': 'enough',
    'faq': 'frequently asked questions',
    'gud': 'good',
    'hai': 'hi',
    'hav': 'have',
    'hbbd': 'happy belated birthday',
    'hbd': 'happy birthday',
    'hf': 'have fun',
    'hru': 'how are you',
    'idc': "i don't care",
    'idk': "i don't know",
    "kwl": "cool",
    'lemeno': 'let me know',
    'laf': 'laugh',
    'min': 'minute',
    'nah': 'no',
    'nvr': 'never',
    'np': 'no problem',
    'nxt': 'next',
    'pls': 'please',
    'oc': 'of course',
    'obv': 'obviously',
    'pic': 'picture',
    'pwd': 'password',
    'rip': 'rest in peace',
    'rm': 'room',
    's2u': 'same to you',
    'sd': 'sweet dreams',
    'sec': 'second',
    'sry': 'sorry',
    'tc': 'take care',
    'tht': 'that',
    'tlk': 'talk',
    'tq': 'thank you',
    'ttyl': 'talk to you later',
    'tyt': 'take your time',
    'tym': 'thank you much',
    'ur': 'your',
    'w/b': 'welcome back',
    'waa': 'crying',
    'wlcm': 'welcome',
    'wtg': 'way to go',
    'wup': "what's up?",
    'ya': 'yes',
    'yeap': 'yes',
    'yep': 'yes',
    'yrs': 'years',
    'yr': 'year',
    'yup': 'yes',
    'zzz': 'sleeping'
}

# Replace tokens in the 'tokenized_tweet' column using the dictionary
train_data['spell_checked_tweet'] = train_data['tokenized_tweet'].replace(abbreviation_dict, regex=True)
train_data['spell_checked_tweet']

0       ['woyould', 'responded', 'mails', 'going', 'co...
1       ['sooo', 'sad', '', 'miss', 'san', 'diego', 'l...
2       ['sorry', 'boss', 'byoullying', 'ceasefire', '...
3       ['interview', 'awesome', 'leave', 'alone', 'do...
4       ['enoyough', 'awkward', 'see', 'becayouse', 'i...
                              ...                        
9994                                              ['too']
9995               ['got', 'byoullied', 'dillah', 'help']
9996                                              ['lol']
9997    ['days', 'myoust', 'look', 'hard', 'good', 'th...
9998    ['worn', 'week', 'sped', 'soo', 'fast', 'gonna...
Name: spell_checked_tweet, Length: 9989, dtype: object

In [120]:
train_data['spell_checked_tweet']

0       ['woyould', 'responded', 'mails', 'going', 'co...
1       ['sooo', 'sad', '', 'miss', 'san', 'diego', 'l...
2       ['sorry', 'boss', 'byoullying', 'ceasefire', '...
3       ['interview', 'awesome', 'leave', 'alone', 'do...
4       ['enoyough', 'awkward', 'see', 'becayouse', 'i...
                              ...                        
9994                                              ['too']
9995               ['got', 'byoullied', 'dillah', 'help']
9996                                              ['lol']
9997    ['days', 'myoust', 'look', 'hard', 'good', 'th...
9998    ['worn', 'week', 'sped', 'soo', 'fast', 'gonna...
Name: spell_checked_tweet, Length: 9989, dtype: object

# 7.) <font color = "black">Using Re</font><a class = "anchor" id = "Re"></a>

In [106]:
def cleaner(sent):
    sent = re.sub("@[A-Za-z0-9]+","",sent) # /@chaitu 
    sent = re.sub(r"(?:\@ |http?\://|https?\://|www)\S+", "", sent)
    sent = " ".join(sent.split())
    sent = sent.replace("#", "").replace("_", " ")
   
    return sent

In [107]:
train_data['tokenized_tweet'] = train_data['tokenized_tweet'].astype(str)
train_data['tokenized_tweet'] = train_data['tokenized_tweet'].apply(cleaner)

In [108]:
train_data['tokenized_tweet'] 

0       ['would', 'responded', 'mails', 'going', 'kwl'...
1       ['sooo', 'sad', '', 'miss', 'san', 'diego', 'l...
2       ['sry', 'boss', 'bullying', 'ceasefire', '', '...
3       ['interview', 'awsm', 'leave', 'alone', 'dnd',...
4              ['enuf', 'awks', 'see', 'bcz', 'insulted']
                              ...                        
9994                                              ['too']
9995                 ['got', 'bullied', 'dillah', 'help']
9996                                              ['lol']
9997    ['days', 'must', 'look', 'hard', 'good', 'thing']
9998    ['worn', 'week', 'sped', 'soo', 'fast', 'gonna...
Name: tokenized_tweet, Length: 9989, dtype: object

In [109]:
train_data['tokenized_tweet'].replace('\d+', '', regex=True, inplace=True)

train_data['tokenized_tweet'].replace('br', '', regex=True, inplace=True)
train_data['tokenized_tweet'].replace('"', '', regex=True, inplace=True)
train_data['tokenized_tweet'].replace('!', '', inplace=True)
train_data['tokenized_tweet'].replace('?', '', inplace=True)
train_data['tokenized_tweet'].replace("-", '', regex=True, inplace=True)
train_data['tokenized_tweet'].replace("*", '', inplace=True)
train_data['tokenized_tweet'].replace("***", '', inplace=True)

train_data['tokenized_tweet'].replace("< />", '', regex=True, inplace=True)


In [110]:
train_data['tokenized_tweet'] 

0       ['would', 'responded', 'mails', 'going', 'kwl'...
1       ['sooo', 'sad', '', 'miss', 'san', 'diego', 'l...
2        ['sry', 'boss', 'bullying', 'ceasefire', '', '']
3       ['interview', 'awsm', 'leave', 'alone', 'dnd',...
4              ['enuf', 'awks', 'see', 'bcz', 'insulted']
                              ...                        
9994                                              ['too']
9995                 ['got', 'bullied', 'dillah', 'help']
9996                                              ['lol']
9997    ['days', 'must', 'look', 'hard', 'good', 'thing']
9998    ['worn', 'week', 'sped', 'soo', 'fast', 'gonna...
Name: tokenized_tweet, Length: 9989, dtype: object

## 8.) Lemmatization:

In [ ]:
import pandas as pd
from nltk.stem import WordNetLemmatizer

# Initialize the WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

# Define a function to perform lemmatization on a list of tokens
def lemmatize_tokens(tokens):
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

# Assuming you already have 'train_data' with a 'tokenized_tweet' column

# Apply lemmatization to the 'tokenized_tweet' column
train_data['tokenized_tweet'] = train_data['tokenized_tweet'].apply(lemmatize_tokens)

## 6.) Text Normalization

In [41]:
my_decontract_dict = {
    "won't": "will not",
    "can't": "can not",
    "n't": " not",
    "'re": " are",
    "'s been": " has been",
    "'s": " is",
    "'d": " would",
    "'ll": " will",
    "'t": " not",
    "ve": " have",
    "'m": " am",
    "im": "i am"
}

In [42]:
train_data['tokenized_tweet']

0       [[, ', w, o, u, l, d, ', ,,  , ', r, e, s, p, ...
1       [[, ', s, o, o, o, ', ,,  , ', s, a, d, ', ,, ...
2       [[, ', b, o, s, s, ', ,,  , ', b, u, l, l, y, ...
3       [[, ', i, n, t, e, r, v, i, e, w, ', ,,  , ', ...
4       [[, ', s, o, n, s, ', ,,  , ', c, o, u, l, d, ...
                              ...                        
9994                                [[, ', t, o, o, ', ]]
9995    [[, ', g, o, t, ', ,,  , ', b, u, l, l, i, e, ...
9996                                [[, ', l, o, l, ', ]]
9997    [[, ', d, a, y, s, ', ,,  , ', m, u, s, t, ', ...
9998    [[, ', w, o, r, n, ', ,,  , ', w, e, e, k, ', ...
Name: tokenized_tweet, Length: 9989, dtype: object

def decontract_text(text):
    for key, value in my_decontract_dict.items():
        text = text.replace(key, value)
    return text

train_data['tokenized_tweet'] = train_data['tokenized_tweet'].apply(decontract_text)
print(train_data['tokenized_tweet'])

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
text_data = train_data['tweet']
tfidf_vectorizer = TfidfVectorizer()
text_data = text_data.fillna("")
tfidf_matrix = tfidf_vectorizer.fit_transform(text_data)
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(data=tfidf_matrix.toarray(), columns=feature_names)

term_scores = tfidf_df.loc[0]  

# Print the TF-IDF scores for each term
for term, score in zip(feature_names, term_scores):
    print(f"{term}: {score}")

00: 0.0
000: 0.0
01: 0.0
02: 0.0
05: 0.0
05ixbj: 0.0
06: 0.0
060: 0.0
07: 0.0
07am: 0.0
07xoh: 0.0
08: 0.0
08kaifj: 0.0
09: 0.0
097dfj: 0.0
0_o: 0.0
0a7v3j: 0.0
0l2tsj: 0.0
0n: 0.0
0rpm: 0.0
0ut: 0.0
0zywwj: 0.0
10: 0.0
100: 0.0
1000: 0.0
10000000000: 0.0
100000000000000000000000000000000000: 0.0
1000th: 0.0
1000x: 0.0
100th: 0.0
1017: 0.0
102: 0.0
1034415: 0.0
105: 0.0
109: 0.0
1095: 0.0
10am: 0.0
10k: 0.0
10mins: 0.0
10pm: 0.0
10th: 0.0
10ty: 0.0
10uemq: 0.0
10yr: 0.0
11: 0.0
1123: 0.0
1130: 0.0
116: 0.0
11e: 0.0
11pm: 0.0
11th: 0.0
12: 0.0
128: 0.0
12am: 0.0
12p: 0.0
12st: 0.0
12th: 0.0
12yr: 0.0
13: 0.0
1313: 0.0
135: 0.0
138: 0.0
13f5m0: 0.0
13nfk1: 0.0
13th: 0.0
14: 0.0
140: 0.0
143: 0.0
144: 0.0
145: 0.0
147heu: 0.0
14m: 0.0
15: 0.0
150: 0.0
1500: 0.0
1500ft: 0.0
1500rpm: 0.0
153bpm: 0.0
15am: 0.0
15c: 0.0
15fo4x: 0.0
15mins: 0.0
15minutes: 0.0
15yyid: 0.0
16: 0.0
160: 0.0
1600th: 0.0
165: 0.0
167: 0.0
1695368792: 0.0
16th: 0.0
17: 0.0
1701: 0.0
1750: 0.0
177: 0.0
17jiy8: 0.0
17

bawas: 0.0
bayou: 0.0
bazillions: 0.0
bazillionz: 0.0
bb: 0.0
bbc: 0.0
bbc1: 0.0
bbl: 0.0
bbl8r: 0.0
bbm: 0.0
bbq: 0.0
bby: 0.0
bbye: 0.0
bbyshower: 0.0
bc: 0.0
bccg: 0.0
bck: 0.0
bcoz: 0.0
bd: 0.0
bday: 0.0
be: 0.0
bea: 0.0
beach: 0.0
beachball: 0.0
beaches: 0.0
beachview: 0.0
beagle: 0.0
beanie: 0.0
beans: 0.0
bear: 0.0
beards: 0.0
bearings: 0.0
bears: 0.0
beast: 0.0
beasted: 0.0
beastie: 0.0
beat: 0.0
beatiful: 0.0
beating: 0.0
beats: 0.0
beatwittyparty: 0.0
beaut: 0.0
beauties: 0.0
beautiful: 0.0
beauty: 0.0
bebe: 0.0
beblessed: 0.0
bebo: 0.0
bec: 0.0
became: 0.0
because: 0.0
beck: 0.0
becky: 0.0
become: 0.0
becomes: 0.0
becoming: 0.0
becos: 0.0
becuz: 0.0
bed: 0.0
bedbury: 0.0
bedroom: 0.0
bedrooms: 0.0
beds: 0.0
bedtime: 0.0
bee: 0.0
beef: 0.0
beeman: 0.0
been: 0.0
beer: 0.0
beers: 0.0
beetle: 0.0
beets: 0.0
before: 0.0
began: 0.0
begged: 0.0
beggin: 0.0
begin: 0.0
begining: 0.0
beginning: 0.0
beginnings: 0.0
begins: 0.0
begun: 0.0
behalf: 0.0
behave: 0.0
behaved: 0.0
behind: 0.0

corporate: 0.0
corral: 0.0
correct: 0.0
corrected: 0.0
correction: 0.0
corrections: 0.0
correctly: 0.0
correspondent: 0.0
corrupted: 0.0
corsen: 0.0
cos: 0.0
cosmic: 0.0
cosmos: 0.0
cost: 0.0
costa: 0.0
costs: 0.0
costsavings: 0.0
costume: 0.0
cote: 0.0
cotton: 0.0
coty: 0.0
couch: 0.0
cougar: 0.0
cough: 0.0
coughing: 0.0
could: 0.0
coulda: 0.0
couldn: 0.0
couldnt: 0.0
counseling: 0.0
count: 0.0
countdown: 0.0
counter: 0.0
counteract: 0.0
counting: 0.0
country: 0.0
counts: 0.0
county: 0.0
coupla: 0.0
couple: 0.0
coupon: 0.0
coupons: 0.0
courage: 0.0
courney: 0.0
course: 0.0
coursework: 0.0
court: 0.0
courtesy: 0.0
courthouse: 0.0
cousin: 0.0
cousins: 0.0
couuuurse: 0.0
cov: 0.0
cover: 0.0
coverage: 0.0
covered: 0.0
covering: 0.0
cow: 0.0
cowboy: 0.0
cowbridge: 0.0
coworker: 0.0
coworkers: 0.0
cox: 0.0
coz: 0.0
cozy: 0.0
cp: 0.0
cple: 0.0
cps: 0.0
cq5nkp: 0.0
cqc: 0.0
cr: 0.0
cr8hjs: 0.0
crab: 0.0
crabs: 0.0
crack: 0.0
crackberry: 0.0
cracked: 0.0
crackerack: 0.0
crackers: 0.0
cracking:

gardening: 0.0
gardens: 0.0
garley: 0.0
garlic: 0.0
garner: 0.0
garnier: 0.0
garrit: 0.0
garth: 0.0
gary: 0.0
gas: 0.0
gash: 0.0
gashes: 0.0
gasp: 0.0
gauss: 0.0
gave: 0.0
gawd: 0.0
gaze: 0.0
gazillion: 0.0
gb: 0.0
gch: 0.0
gd: 0.0
gda: 0.0
ge: 0.0
gear: 0.0
gearbox: 0.0
geas: 0.0
geee: 0.0
geek: 0.0
geeky: 0.0
geelong: 0.0
geez: 0.0
gelli: 0.0
gem: 0.0
gender: 0.0
general: 0.0
generalisations: 0.0
generation: 0.0
genetic: 0.0
geneva: 0.0
genghis: 0.0
genious: 0.0
genitals: 0.0
genius: 0.0
genre: 0.0
gents: 0.0
genuine: 0.0
genuinely: 0.0
geography: 0.0
geometry: 0.0
geordanos: 0.0
george: 0.0
georgia: 0.0
gerbil: 0.0
german: 0.0
germany: 0.0
germination: 0.0
germs: 0.0
get: 0.0
getcho: 0.0
gets: 0.0
gettin: 0.0
getting: 0.0
gettn: 0.0
getttin: 0.0
gf: 0.0
gfs: 0.0
gg: 0.0
ghalib: 0.0
ghetto: 0.0
ghey: 0.0
ghost: 0.0
ghosts: 0.0
ghostwhisperer: 0.0
giant: 0.0
giants: 0.0
gifford: 0.0
gift: 0.0
gifts: 0.0
gig: 0.0
gigantic: 0.0
giggle: 0.0
gigs: 0.0
gigwise: 0.0
gihac: 0.0
giid: 0.0
gil

kairos: 0.0
kaka: 0.0
kaki: 0.0
kalebnation: 0.0
kaleidoscope: 0.0
kaliaaannn: 0.0
kamis: 0.0
kamusta: 0.0
kan: 0.0
kane: 0.0
kangaroo: 0.0
kanin: 0.0
kanina: 0.0
kanoa: 0.0
kansas: 0.0
kapag: 0.0
kapsel: 0.0
kaput: 0.0
karan: 0.0
karaoke: 0.0
karasw: 0.0
karate: 0.0
kari: 0.0
karma: 0.0
kashmir: 0.0
kat: 0.0
kate: 0.0
katey: 0.0
kathy: 0.0
katie: 0.0
katnip: 0.0
katy: 0.0
kauai: 0.0
kavya: 0.0
kay: 0.0
kayla: 0.0
kaylee: 0.0
kayleigh: 0.0
kaylen: 0.0
kayo: 0.0
kayy: 0.0
kazim: 0.0
kb: 0.0
kbangun: 0.0
kc: 0.0
ke: 0.0
keane: 0.0
keegan: 0.0
keemie: 0.0
keen: 0.0
keep: 0.0
keeping: 0.0
keeps: 0.0
keith: 0.0
kel: 0.0
kelantan: 0.0
keliling: 0.0
kelli: 0.0
kellie: 0.0
kelly: 0.0
kellynn: 0.0
ken: 0.0
kenan: 0.0
kendra: 0.0
kenny: 0.0
kept: 0.0
kerbear: 0.0
kerk: 0.0
kernel: 0.0
keshia: 0.0
keswick: 0.0
ketchup: 0.0
kettle: 0.0
keve: 0.0
kevin: 0.0
kewl: 0.0
key: 0.0
keyboard: 0.0
keyboards: 0.0
keyed: 0.0
keyhole: 0.0
keyla: 0.0
keynote: 0.0
keypad: 0.0
keys: 0.0
keyword: 0.0
kh: 0.0
khak

nicer: 0.0
nichi: 0.0
nicho: 0.0
nicholas: 0.0
nick: 0.0
nicked: 0.0
nicks: 0.0
nicky: 0.0
nico: 0.0
nicole: 0.0
nicoles: 0.0
nicotine: 0.0
niece: 0.0
nielsenpowermoms: 0.0
nige: 0.0
nigel: 0.0
niggas: 0.0
nigguh: 0.0
night: 0.0
nightall: 0.0
nighter: 0.0
nightly: 0.0
nightmare: 0.0
nightmares: 0.0
nights: 0.0
nightshift: 0.0
nightt: 0.0
nighttt: 0.0
nighty: 0.0
nih: 0.0
nike: 0.0
nikeplus: 0.0
niko: 0.0
nikon: 0.0
nila: 0.0
niley: 0.0
nimorning: 0.0
nims: 0.0
nin: 0.0
nina: 0.0
nine: 0.0
nineteenth: 0.0
ning: 0.0
nintendo: 0.0
nipissing: 0.0
nirvana: 0.0
nishiki: 0.0
nita: 0.0
nite: 0.0
njoying: 0.0
njubcl: 0.0
nkorea: 0.0
nkotb: 0.0
nkotbtweetentixsdeal: 0.0
nl: 0.0
nm: 0.0
nnc: 0.0
nnnaaa: 0.0
no: 0.0
no1: 0.0
nobles: 0.0
noboby: 0.0
nobody: 0.0
nocturnal: 0.0
nod: 0.0
nodding: 0.0
nods: 0.0
noe: 0.0
noes: 0.0
nofakery: 0.0
nogard: 0.0
noida: 0.0
noise: 0.0
noises: 0.0
noisy: 0.0
nom: 0.0
nomore: 0.0
noms: 0.0
non: 0.0
none: 0.0
nonetheless: 0.0
nonfunctionalproductive: 0.0
nonsense

ro: 0.0
road: 0.0
roads: 0.0
roadtrip: 0.0
roadtrips: 0.0
roar: 0.0
roast: 0.0
roasted: 0.0
roasting: 0.0
rob: 0.0
robbed: 0.0
robin: 0.0
robina: 0.0
robins: 0.0
robinson: 0.0
robluketic: 0.0
robots: 0.0
robs: 0.0
roc: 0.0
rochester: 0.0
rocio: 0.0
rock: 0.0
rocked: 0.0
rocket: 0.0
rockin: 0.0
rocking: 0.0
rockpools: 0.0
rocks: 0.0
rockstar: 0.0
rocky: 0.0
rocstar: 0.0
rodney: 0.0
rof: 0.0
rofl: 0.0
roflmao: 0.0
rogers: 0.0
rohan: 0.0
role: 0.0
rolf: 0.0
roll: 0.0
rolled: 0.0
roller: 0.0
rollerblading: 0.0
rollerskate: 0.0
rolling: 0.0
rollo: 0.0
rolls: 0.0
rolynn719: 0.0
romance: 0.0
romania: 0.0
romantic: 0.0
rome: 0.0
romeo: 0.0
romina: 0.0
romo: 0.0
romulin: 0.0
ron: 0.0
roni: 0.0
ronin121: 0.0
roo: 0.0
roof: 0.0
rooftop: 0.0
room: 0.0
roomate: 0.0
roomie: 0.0
roomies: 0.0
roommate: 0.0
roommates: 0.0
rooms: 0.0
roooooooooom: 0.0
root: 0.0
rooting: 0.0
roots: 0.0
rope: 0.0
rosa: 0.0
rosary: 0.0
roscoe: 0.0
rose: 0.0
roseanne: 0.0
rosemary: 0.0
roses: 0.0
ross: 0.0
rotation: 0.0
rot

thirsty: 0.0
thirteen: 0.0
thirty: 0.0
this: 0.0
thks: 0.0
thm: 0.0
thnks: 0.0
thnx: 0.0
tho: 0.0
thomas: 0.0
thooo: 0.0
thornberrys: 0.0
those: 0.0
thou: 0.0
though: 0.0
thought: 0.0
thoughtful: 0.0
thoughts: 0.0
thousand: 0.0
thousands: 0.0
thr: 0.0
thread: 0.0
threads: 0.0
threatening: 0.0
threatens: 0.0
three: 0.0
threshold: 0.0
threw: 0.0
thrilled: 0.0
thrilling: 0.0
throat: 0.0
throbbing: 0.0
through: 0.0
throw: 0.0
throwback: 0.0
throwdown: 0.0
throwing: 0.0
thrown: 0.0
thru: 0.0
ths: 0.0
tht: 0.0
thts: 0.0
thumbnail: 0.0
thumbs: 0.0
thunder: 0.0
thundered: 0.0
thundering: 0.0
thunderrrr: 0.0
thundershowers: 0.0
thunderstorm: 0.0
thunderstorming: 0.0
thunderstorms: 0.0
thurs: 0.0
thursday: 0.0
thus: 0.0
thx: 0.0
thxs: 0.0
ti: 0.0
ticker: 0.0
ticket: 0.0
tickets: 0.0
tickettttt: 0.0
ticketttttss: 0.0
tickled: 0.0
tide: 0.0
tidied: 0.0
tidy: 0.0
tie: 0.0
tied: 0.0
tierd: 0.0
ties: 0.0
tiesto: 0.0
tiff: 0.0
tigerheat: 0.0
tight: 0.0
tighten: 0.0
tiiiiiiired: 0.0
tiiiiiimmmmmmeee: 0

½re: 0.0
½s: 0.0
½sseldorf: 0.0
½sst: 0.0
½t: 0.0
½ve: 0.0
½you: 0.0
½ï: 0.0
